In [353]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# Any results you write to the current directory are saved as output.

/kaggle/input/google-cloud-ncaa-march-madness-2020-division-1-mens-tournament/MEvents2016.csv
/kaggle/input/google-cloud-ncaa-march-madness-2020-division-1-mens-tournament/MEvents2018.csv
/kaggle/input/google-cloud-ncaa-march-madness-2020-division-1-mens-tournament/MEvents2015.csv
/kaggle/input/google-cloud-ncaa-march-madness-2020-division-1-mens-tournament/MEvents2019.csv
/kaggle/input/google-cloud-ncaa-march-madness-2020-division-1-mens-tournament/MEvents2017.csv
/kaggle/input/google-cloud-ncaa-march-madness-2020-division-1-mens-tournament/MSampleSubmissionStage1_2020.csv
/kaggle/input/google-cloud-ncaa-march-madness-2020-division-1-mens-tournament/MPlayers.csv
/kaggle/input/google-cloud-ncaa-march-madness-2020-division-1-mens-tournament/MDataFiles_Stage1/Cities.csv
/kaggle/input/google-cloud-ncaa-march-madness-2020-division-1-mens-tournament/MDataFiles_Stage1/MTeamCoaches.csv
/kaggle/input/google-cloud-ncaa-march-madness-2020-division-1-mens-tournament/MDataFiles_Stage1/MTeamSpellin

In [354]:
import pandas as pd
import numpy as np
import os
import sys

import re
import random
import matplotlib.pyplot as plt
from sklearn.utils import shuffle
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import StratifiedKFold
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import KFold
import lightgbm as lgb
import xgboost as xgb
from xgboost import XGBClassifier
import gc
import glob

%matplotlib inline

In [355]:
df = {}
for i in glob.glob('/kaggle/input/google-cloud-ncaa-march-madness-2020-division-1-mens-tournament/MDataFiles_Stage1/*'):
    name = i.split('/')[-1].split('.')[0]
    if name != 'MTeamSpellings':
        df[name] = pd.read_csv(i)
    else:
        df[name] = pd.read_csv(i, encoding='cp1252')

In [356]:
df.keys()

dict_keys(['Cities', 'MTeamCoaches', 'MTeamSpellings', 'MMasseyOrdinals', 'MSeasons', 'MTeams', 'MSecondaryTourneyTeams', 'MNCAATourneyCompactResults', 'MGameCities', 'Conferences', 'MNCAATourneySeeds', 'MNCAATourneySlots', 'MTeamConferences', 'MNCAATourneySeedRoundSlots', 'MNCAATourneyDetailedResults', 'MConferenceTourneyGames', 'MRegularSeasonDetailedResults', 'MRegularSeasonCompactResults', 'MSecondaryTourneyCompactResults'])

In [357]:
regular=df['MRegularSeasonCompactResults']
regular

,Season,DayNum,WTeamID,WScore,LTeamID,LScore,WLoc,NumOT
0,1985,20,1228,81,1328,64,N,0
1,1985,25,1106,77,1354,70,H,0
2,1985,25,1112,63,1223,56,H,0
3,1985,25,1165,70,1432,54,H,0
4,1985,25,1192,86,1447,74,H,0
...,...,...,...,...,...,...,...,...
161547,2019,132,1153,69,1222,57,N,0
161548,2019,132,1209,73,1426,64,N,0
161549,2019,132,1277,65,1276,60,N,0
161550,2019,132,1387,55,1382,53,N,0


In [358]:
tourney=df['MNCAATourneyCompactResults']
tourney

,Season,DayNum,WTeamID,WScore,LTeamID,LScore,WLoc,NumOT
0,1985,136,1116,63,1234,54,N,0
1,1985,136,1120,59,1345,58,N,0
2,1985,136,1207,68,1250,43,N,0
3,1985,136,1229,58,1425,55,N,0
4,1985,136,1242,49,1325,38,N,0
...,...,...,...,...,...,...,...,...
2246,2019,146,1120,77,1246,71,N,1
2247,2019,146,1277,68,1181,67,N,0
2248,2019,152,1403,61,1277,51,N,0
2249,2019,152,1438,63,1120,62,N,0


In [359]:
secondary=df['MSecondaryTourneyCompactResults']
secondary

,Season,DayNum,WTeamID,WScore,LTeamID,LScore,WLoc,NumOT,SecondaryTourney
0,1985,136,1151,67,1155,65,H,0,NIT
1,1985,136,1153,77,1245,61,H,0,NIT
2,1985,136,1201,79,1365,76,H,0,NIT
3,1985,136,1231,79,1139,57,H,0,NIT
4,1985,136,1249,78,1222,71,H,0,NIT
...,...,...,...,...,...,...,...,...,...
1619,2019,148,1453,87,1411,86,H,1,CIT
1620,2019,149,1177,100,1378,96,H,1,CBI
1621,2019,150,1267,90,1453,70,H,0,CIT
1622,2019,150,1400,81,1252,66,N,0,NIT


In [360]:
secondary.drop('SecondaryTourney',axis=1,inplace=True)
secondary

,Season,DayNum,WTeamID,WScore,LTeamID,LScore,WLoc,NumOT
0,1985,136,1151,67,1155,65,H,0
1,1985,136,1153,77,1245,61,H,0
2,1985,136,1201,79,1365,76,H,0
3,1985,136,1231,79,1139,57,H,0
4,1985,136,1249,78,1222,71,H,0
...,...,...,...,...,...,...,...,...
1619,2019,148,1453,87,1411,86,H,1
1620,2019,149,1177,100,1378,96,H,1
1621,2019,150,1267,90,1453,70,H,0
1622,2019,150,1400,81,1252,66,N,0


In [361]:
all_match=pd.concat((regular,tourney,secondary))
all_match.sort_values(by=['Season','DayNum'],inplace=True)
all_match.drop(['DayNum','NumOT'],axis=1,inplace=True)
all_match

,Season,WTeamID,WScore,LTeamID,LScore,WLoc
0,1985,1228,81,1328,64,N
1,1985,1106,77,1354,70,H
2,1985,1112,63,1223,56,H
3,1985,1165,70,1432,54,H
4,1985,1192,86,1447,74,H
...,...,...,...,...,...,...
1622,2019,1400,81,1252,66,N
1623,2019,1378,77,1177,65,A
2248,2019,1403,61,1277,51,N
2249,2019,1438,63,1120,62,N


In [362]:

all_match.groupby('WLoc').size()


WLoc
A    49701
H    96929
N    18797
dtype: int64

In [363]:
seeds=df['MNCAATourneySeeds']
seeds

,Season,Seed,TeamID
0,1985,W01,1207
1,1985,W02,1210
2,1985,W03,1228
3,1985,W04,1260
4,1985,W05,1374
...,...,...,...
2281,2019,Z12,1332
2282,2019,Z13,1414
2283,2019,Z14,1330
2284,2019,Z15,1159


In [364]:
seeds['Seed'] = seeds['Seed'].apply(lambda x: int(x[1:3]))
seeds

,Season,Seed,TeamID
0,1985,1,1207
1,1985,2,1210
2,1985,3,1228
3,1985,4,1260
4,1985,5,1374
...,...,...,...
2281,2019,12,1332
2282,2019,13,1414
2283,2019,14,1330
2284,2019,15,1159


In [365]:
data=pd.merge(all_match,seeds,how='left',left_on=['Season','WTeamID'],right_on=['Season','TeamID'])
data.drop(['TeamID'],axis=1,inplace=True)
data.rename(columns={'Seed':'Wseed'},inplace=True)

data=pd.merge(data,seeds,how='left',left_on=['Season','LTeamID'],right_on=['Season','TeamID'])
data.drop(['TeamID'],axis=1,inplace=True)
data.rename(columns={'Seed':'Lseed'},inplace=True)

data=data[['Season','WTeamID','LTeamID','Wseed','Lseed','WScore','LScore','WLoc']]
data

,Season,WTeamID,LTeamID,Wseed,Lseed,WScore,LScore,WLoc
0,1985,1228,1328,3.0,1.0,81,64,N
1,1985,1106,1354,NaN,NaN,77,70,H
2,1985,1112,1223,10.0,NaN,63,56,H
3,1985,1165,1432,NaN,NaN,70,54,H
4,1985,1192,1447,16.0,NaN,86,74,H
...,...,...,...,...,...,...,...,...
165422,2019,1400,1252,NaN,NaN,81,66,N
165423,2019,1378,1177,NaN,NaN,77,65,A
165424,2019,1403,1277,3.0,2.0,61,51,N
165425,2019,1438,1120,1.0,5.0,63,62,N


In [366]:
data.isna().sum()
data.dropna(axis=0,inplace=True)#delete NaN row
data.reset_index(drop=True,inplace=True)

data.rename(columns={'WTeamID':'TeamA','LTeamID':'TeamB'},inplace=True)
data.rename(columns={'WScore':'AScore','LScore':'BScore'},inplace=True)
data.rename(columns={'Wseed':'ASeed','Lseed':'BSeed','WLoc':'Loc'},inplace=True)
data

,Season,TeamA,TeamB,ASeed,BSeed,AScore,BScore,Loc
0,1985,1228,1328,3.0,1.0,81,64,N
1,1985,1242,1268,3.0,5.0,58,56,N
2,1985,1305,1424,14.0,4.0,97,89,H
3,1985,1412,1228,7.0,3.0,59,52,N
4,1985,1439,1330,9.0,12.0,102,76,H
...,...,...,...,...,...,...,...,...
12006,2019,1120,1246,5.0,2.0,77,71,N
12007,2019,1277,1181,2.0,1.0,68,67,N
12008,2019,1403,1277,3.0,2.0,61,51,N
12009,2019,1438,1120,1.0,5.0,63,62,N


In [367]:
win=data.copy()
lose=data.copy()

lose['TeamA']=win['TeamB']
lose['TeamB']=win['TeamA']

lose['ASeed']=win['BSeed']
lose['BSeed']=win['ASeed']

lose['AScore']=win['BScore']
lose['BScore']=win['AScore']

lose['Loc']=lose['Loc'].apply(lambda x: 'H' if x=='N' else 'N' if x=='H' else 'A')

win['result']=1
lose['result']=0

total=pd.concat((win,lose))
total['Loc']=total['Loc'].apply(lambda x: '1' if x=='H' else '0' if x=='N' else '-1')

total['seed_diff']=total['ASeed']-total['BSeed']
total['score_diff']=total['AScore']-total['BScore']

train=total[['Season','TeamA','TeamB','ASeed','BSeed','AScore','BScore','seed_diff','score_diff','Loc','result']]
train


,Season,TeamA,TeamB,ASeed,BSeed,AScore,BScore,seed_diff,score_diff,Loc,result
0,1985,1228,1328,3.0,1.0,81,64,2.0,17,0,1
1,1985,1242,1268,3.0,5.0,58,56,-2.0,2,0,1
2,1985,1305,1424,14.0,4.0,97,89,10.0,8,1,1
3,1985,1412,1228,7.0,3.0,59,52,4.0,7,0,1
4,1985,1439,1330,9.0,12.0,102,76,-3.0,26,1,1
...,...,...,...,...,...,...,...,...,...,...,...
12006,2019,1246,1120,2.0,5.0,71,77,-3.0,-6,1,0
12007,2019,1181,1277,1.0,2.0,67,68,-1.0,-1,1,0
12008,2019,1277,1403,2.0,3.0,51,61,-1.0,-10,1,0
12009,2019,1120,1438,5.0,1.0,62,63,4.0,-1,1,0


test data

In [368]:
test=pd.read_csv('/kaggle/input/google-cloud-ncaa-march-madness-2020-division-1-mens-tournament/MSampleSubmissionStage1_2020.csv')
test

,ID,Pred
0,2015_1107_1112,0.5
1,2015_1107_1116,0.5
2,2015_1107_1124,0.5
3,2015_1107_1125,0.5
4,2015_1107_1129,0.5
...,...,...
11385,2019_1449_1459,0.5
11386,2019_1449_1463,0.5
11387,2019_1458_1459,0.5
11388,2019_1458_1463,0.5


In [369]:
test[['Season','A','B']]=test['ID'].str.split('_',expand=True)
test['Season']=test['Season'].astype(int)
test['A']=test['A'].astype(int)
test['B']=test['B'].astype(int)
test.drop(['ID','Pred'],axis=1,inplace=True)
test

Season    int64
A         int64
B         int64
dtype: object

In [370]:
test=pd.merge(test,seeds,how='left',left_on=['Season','A'],right_on=['Season','TeamID'])
test

,Season,A,B,Seed,TeamID
0,2015,1107,1112,14,1107
1,2015,1107,1116,14,1107
2,2015,1107,1124,14,1107
3,2015,1107,1125,14,1107
4,2015,1107,1129,14,1107
...,...,...,...,...,...
11385,2019,1449,1459,9,1449
11386,2019,1449,1463,9,1449
11387,2019,1458,1459,5,1458
11388,2019,1458,1463,5,1458
